In [ ]:
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
from mpmath import findroot, cosh, sinh
from scipy.optimize import fsolve

# General  integrate-and-fire neuron

In gernal, the dynamics of a integrate-and-fire (IF) neuron can be described by 

\begin{equation}
\frac{dV}{dt}=f(V)+I(V,t)
\end{equation}

where f(V) is the deterministic drift towards the resting potential. E.g. for a 

-  Leaky IF neuron: &nbsp;$f(V)=-V/\tau$
-  Perfect IF neuron: &nbsp; $f(V)=-\beta$ 

In the diffusion approximation, we assume that $I(V,t)$ can replaced by Gaussian white noise. Hence, we are arrive at a stochastic description and the dynamics which are governed by a Langevin equation  

\begin{equation}
\frac{dv}{dt}=h(v,t)+g(v,t)\Gamma(t)
\end{equation}

where $h(v,t)$ is the sum of the deterministic drift plus the average afferent current $\mu(v,t)$ and $g(v,t)$ is the size of the current fluctuations. $\Gamma(t)$ is a Gaussian whites noise with zero mean and unitary variance. For a population, we  assume that the statistics of the afferent current at a given time are identical for all neurons. Furthermore, we assume that the noise feld by each neuron is statistically independent. This is called the mean-field assumption.<br> 
From the Langevin equation, we can derive a Fokker-Planck equation for density $\rho(v,t)$

\begin{equation}
\partial_{t}\rho(v,t)=L\rho(v,t)
\end{equation}

where $L$ is the Fokker-Planck operator 

\begin{equation}
L = -\partial_{v}h(v,t)+\frac{1}{2}\partial_{v}^{2}g(v,t)
\end{equation}

The Fokker-Planck equation can be written as a continuity equation 

\begin{equation}
\partial_{t}\rho=-\partial_{v}J,
\end{equation}

where $J$ is the probability flux 

\begin{equation}
J = (h(v,t)-\frac{1}{2}\partial_{v}g(v,t)) \rho(v,t).
\end{equation}

Spike generation can be incorporated into the model via boundary conditions. After a neuron emits a spike, the membrane potential is clamped to reset for the refractory period $t_0$. If the probability mass reaches the threshold $v=\theta$ it is reinserted at the reset potential $v=H$. This behavior can be modeled by a absorbing boundary, i.e. $\rho(v \geq \theta, t) = 0$. Because the probability mass crossing the threshold is reinserted at reset, the flux makes a jump at $v=H$.

\begin{equation}
J(\theta, t - t_0) = J(H_{+}, t) - J(H_{-}, t) 
\end{equation}

The above equation essentially means that the flux $J(H_{+}, t)$ at right of $H$ is the sum of the flux coming from the left at $H$ and the flux which passes the threshold and gets reinserted. 

We may also define a lower bound $v_{\text{min}}$ (possibly $v_{\text{min}}=-\infty$) for the membrane potential which can be modelled by a reflecting boundary, i.e. $J(v_{\text{min}}, t) = 0$. To conclude, we have three different boundary conditions  

1. &nbsp; $\rho(v \geq \theta, t) = 0$
2. &nbsp; $J(\theta, t - t_0) = J(H_{+}, t) - J(H_{-}, t)$
3. &nbsp; $J(v \leq v_{\text{min}}, t) = 0$

The time-dependence of the Fokker-Planck operator is implicit through the time dependence of the input  statistics. For any specified input, the Fokker-Planck operator will have a set of eigenvectors $\phi_n$ with associated eigenvalues $\lambda_n$ such that 

\begin{equation}
L\phi_{n}=\lambda_{n}\phi_{n}
\end{equation}

where $\phi$ satisfies the previously defined boundary conditions. $L$, $\phi_n$ and $\lambda_n$ all depend on the input statistics, and therefore on time 

\begin{equation}
L=L(t),\,\phi_{n}=\phi_{n}(t),\,\lambda_{n}=\lambda_{n}(t)
\end{equation}

Note that the eigenfunctions $\phi_{n}$ are not orthogonal, because $L$ is not Hermitian. We define the inner product

\begin{equation}
\bigl\langle\psi,\phi\bigr\rangle=\int dv\,\psi(v)\phi(v)
\end{equation}

and the adjoint operator $L^{\dagger}$ such that  

\begin{equation}
\bigl\langle\psi,L\phi\bigr\rangle=\bigl\langle L^{\dagger}\psi,\phi\bigr\rangle
\end{equation}

Using the above definition, we find the adjoint operator

\begin{equation}
L^{\dagger}=h(v,t)\partial_{v}+\frac{1}{2}g(v,t)\partial_{v}^{2},
\end{equation}

and the boundary conditions for the adjoint eigenfunctions $\psi_n$

1. &nbsp; $\psi_{n}(\theta,t)S_{\phi_{n}}(\theta,t)=\psi_{n}(H,t)S_{\phi_{n}}(\theta,t-\tau_{0})$  
2. &nbsp; $\partial_{v}\psi_{n}(v_{\text{min}},t)=0$
3. &nbsp; $\partial_{v}\psi_{n}(H_{+},t)=\partial_{v}\psi_{n}(H_{-},t)$

## Eigenfunction expansion

Pairs of eigenfunctions $(\psi_{n}, \phi_{n})$ span a complete bi-orthogonal basis  

\begin{equation}
\sum_{n}\phi_{n}(v)\psi_{n}(v')=\delta(v-v')
\end{equation}

so that we can expand the probability density in terms of eigenfunctions 

\begin{equation}
\rho(v)  =\sum_{n} a_{n} \phi_{n}(v),
\end{equation}

where the coefficients $a_{n}$ are given by  

\begin{equation}
a_{n}=\int dv\,\psi_{n}(v)\rho(v)=\bigl\langle\psi_{n},\,\rho\bigr\rangle
\end{equation}

Using the above equation, the dynamics of the coefficients are described by the   

\begin{equation}
\partial_t a = \partial_t \bigl\langle\psi_{n},\,\rho\bigr\rangle =\bigl\langle\psi_{n}, \partial_{t} \rho \bigr\rangle+ \bigl\langle\ \partial_{t} \psi_n, \rho\bigr\rangle 
\end{equation}

Substituting $\partial_{t} \rho = L\rho$ and expression $\rho$ in terms of the eigenfunctions $\phi_n$ yields   

\begin{equation}
\dot{a}_{n}=	\sum_{m}\left(\lambda_{n}a_{m}\bigl\langle\psi_{n},\phi_{m}\bigr\rangle+a_{m}\bigl\langle\partial_{t}\psi_{n},\phi_{m}\bigr\rangle\right)=\lambda_{n}a_{n}+\sum_{m}a_{m}\bigl\langle\partial_{t}\psi_{n},\phi_{m}\bigr\rangle
\end{equation}

The coupling term contains the derivatives $\partial_{t}\psi_{n}$. Note that $\psi_{n}$ depends on time due to the time dependence of the Fokker-Planck operator. The time dependence of the Fokker-Planck operator is implicit through the rate $r(t)$ which enters appears in the drift and diffusion coefficient so that  

\begin{equation}
\dot{a}_{n}=\lambda_{n}a_{n}+ \dot{r} \sum_{m}a_{m}\bigl\langle \partial_{r}\psi_{n},\phi_{m}\bigr\rangle
\end{equation}

We distinguish between the emission rate $r_{\text{e}}(t)$ a neuron feels due recurrent connections from other neurons in the population and the external rate $r_{\text{ext}}(t)$ due to connections from neurons in the external bath surrounding the population so that 

\begin{equation}
r(t)=r_{\text{e}}(t)+r_{\text{ext}}(t).
\end{equation}

The average emission rate at a given time is equivalent to the probality flux crossing threshold $r_{\text{e}}(t)=J(\theta,t - t_{0})$. Inserting the definition of $J$ and using that $\rho(\theta, t)$ vanishes at the boundary, we arrive at 

\begin{equation}
r_{\text{e}}(t)=J(\theta,t-t_{0})=\left.-\frac{1}{2}\partial_{v}g(v,t-t_{0}))\rho(v,t-t_{0})\right|_{v=\theta}
\end{equation}

Expanding $\rho(v,t-t_{0})$ in terms of eigenfunctions yields 

\begin{equation}
r_{\text{e}}(t)=\left.-\frac{1}{2}\sum_{n}a_{n}(t-t_{0})\partial_{v}g(v,t-t_{0})\phi_{n}(v,t-t_{0})\right|_{v=\theta}
\end{equation}

It is convenient to introduce a notation which separates the zero (steady-state) mode from the nonstationary higher modes. We define  

\begin{equation}
r_{\text{e}}^{(0)}(t) =\left.-\frac{1}{2}\partial_{v}g(v,t-t_{0})\phi_{0}(v,t-t_{0})\right|_{v=\theta}
\end{equation}

where we used $a_{0}(t) = 1$. In the same spirit, we can write

\begin{equation}
\dot{a}_{n} = \lambda_{n}a_{n}+\dot{r} a_{0}\bigl\langle\partial_{r}\psi_{n},\phi_{0}\bigr\rangle+\dot{r}\sum_{m=1}a_{m}\bigl\langle\partial_{r}\psi_{n},\phi_{m}\bigr\rangle
\end{equation}

Defining the coefficient vector $\boldsymbol{a}=(a_{1},a_{2}, \ldots)$, the above equation can be written in matrix notation 

\begin{equation} \label{eq: a matrix}
\dot{\boldsymbol{a}} =\left(\boldsymbol{\Lambda}+\boldsymbol{C}\dot{r}\right)\boldsymbol{a}+\boldsymbol{c}\dot{r}
\end{equation}

where $\boldsymbol{\Lambda}$ is a diagonal matrix with 

\begin{equation}
\Lambda_{nm}=\lambda_{n}\delta_{nm},
\end{equation}

$\boldsymbol{C}$ the coupling matrix between nonstationary modes 

\begin{equation}
C_{nm}=\bigl\langle\partial_{r}\psi_{n}, \phi_{m}\bigr\rangle,
\end{equation}

and the elements of vector $\boldsymbol{c}$ are the coupling terms between the nonstationary modes 

\begin{equation}
c_{n}=\left\langle \partial_{r}\psi_{n},\phi_{0}\right\rangle.
\end{equation}

The rate equation can be written as 

\begin{equation}
r=r_{\text{e}}^{(0)}+r_{\text{e}}^{(n>0)}+r_{\text{ext}}
\end{equation}

where $r_{\text{e}}^{(n>0)}$ are contributions to the emission rate due to nonstationary modes. It can be written as a scalar product  

\begin{equation}
r_{\text{e}}^{(n>0)} = \boldsymbol{f}\boldsymbol{a}
\end{equation}

where we defined the vector $\boldsymbol{f}$ whose elements are the flux over boundary of nonstationary modes 

\begin{equation}
f_{n}=\left. -\frac{1}{2} g(v,t-t_{0})\partial_{v}\phi_{n}(v,t-t_{0})\right|_{v=\phi}.
\end{equation}

Assuming that a finite and small number of higher modes is sufficient to describe the time evolution of $r$. This ansatz provides an effective way to reduce the dimensionality of the problem. To use the method, we need to know the eigenvalues and eigenfunctions of the Fokker-Planck operator in dependence of the rate. 

## Non-interacting-neurons 

For noninteracting neurons, there are no recurrent connections, i.e. the input rate is equivalent to the rate of the external bath 

\begin{equation}
r(t) = r_{\text{ext}}(t)
\end{equation}

If we assume, that $r_{\text{ext}}(t)$ is stationary, then matrix equation \ref{eq: a matrix} diagonlizes 

\begin{equation}
\dot{\boldsymbol{a}} = \boldsymbol{\Lambda}\boldsymbol{a},
\end{equation}

because $\dot{r}=0$. The emission rate is given by 

\begin{equation}

\end{equation}

# Perfect IF neuron

We apply the decomposition to perfect integrate-and-fire neurons 

\begin{equation}
\frac{dV}{dt}=-\beta+I(V,t).
\end{equation}

In the following, we assume that the average afferent current $\mu(v,t)$ and the size of the current fluctuations $g(v,t)$ do not depend on the membrane potential. Hence, the Fokker-Planck operator at time $t$ is given by 

\begin{equation}
L =-h(t)\partial_{v}+\frac{1}{2}g(t)\partial_{v}^{2}.
\end{equation}

The drift coefficient $h(t)$ and the diffusion coefficient are given by 

\begin{equation}
h(t)=\mu(t)-\beta,\quad g(t)=\sigma^{2}(t)
\end{equation}

with $\mu(t) = h\tau r(t)$ and $\sigma^2(t) = h^2 \tau r(t)$. We define $\eta(t) = \mu(t) - \beta$.



## Eigenfunctions

The eigenfunctions $\phi_n(v)$ fulfill $L\phi_{n}=\lambda_{n}\phi_{n}$ which leads to a second order ODE with constant coefficients   

\begin{equation}
-h\partial_{v}\phi_{n}(v)+\frac{1}{2}g\partial_{v}^{2}\phi_{n}(v)=\lambda_{n}\phi_{n}(v)
\end{equation}

Note that we supressed the time arguments. The ansatz $\phi_{n}(v)=e^{\alpha_{n} v}$ yields the general solution

\begin{equation}
\phi_{n}(v)=c_{1} e^{\alpha_{1}v}+c_{2}e^{\alpha_{2}v}
\end{equation}

where $\alpha_{1}$ and $\alpha_{2}$ are given by 

\begin{equation}
\alpha_{1/2} = \pm\frac{1}{\sigma^2}\sqrt{2\lambda_{n} \sigma^2+h^{2}}+\frac{\eta}{\sigma^2}
\end{equation}

We define $\zeta=\frac{\theta}{\sigma^2}\sqrt{2\lambda_{n} \sigma^2+ \eta^{2}}$ and $\xi=\frac{\theta h}{\sigma^2}$ so that 

\begin{equation}
\alpha_{1/2} = \frac{\pm \zeta + \xi}{\theta}
\end{equation}

Hence, the general solution is given by

\begin{equation}
\phi_{n}(v)= \left(c_{1}e^{\frac{\zeta}{\theta}v}+c_{2}e^{\frac{-\zeta}{\theta}v}\right)e^{\frac{\xi}{\theta}v}
\end{equation}

### Zero Eigenmode 
 
Setting $\lambda_0=0$ yields the zero mode eigenfunction 

\begin{equation}
\phi_{0}(v,t)=\frac{c}{\eta}\left(1-e^{\frac{-2\xi(\theta-v)}{\theta}}\right)
\end{equation}

with constant 
\begin{equation}
c^{-1}=\frac{\sigma^{2}}{2\eta^{2}}\left(2\xi-1+e^{-2\xi}\right)
\end{equation}

The stationary emission rate is given by the flux over threshold 

\begin{equation}
r_{\text{e}}^{(0)} = \frac{1}{2}\sigma^{2}\partial_{v}\phi_{0}(\theta)=c
\end{equation}

We define 

\begin{equation}
\Phi(\eta,\sigma^{2})= c(\eta, \sigma^2)
\end{equation}

### Higher modes

The coefficients $c_{1}$ and $c_{2}$ needs to be determined from the boundary conditions. From $\phi(\theta,t)=0$ follows 

\begin{equation}
\phi_{n}(v) = c_n \sinh\left(\zeta_n\frac{\theta_n - v}{\theta}\right)e^{\frac{\xi v}{\theta}}
\end{equation}

Note that $\zeta_n$ depends on the eigenvalue $\lambda_n$. Using the same ansatz and the boundary conditions for the adjoint eigenfunctions, we find 

\begin{equation}
\psi_{n}(v,t)=\left[\zeta_n \cosh\frac{\zeta_n v}{\theta}+\xi\sinh\frac{\zeta_n v}{\theta}\right]e^{-\frac{\xi}{\theta}v}
\end{equation}

Here, we absorbed the undetermined coefficient into $c_n$. The coefficient $c_n$ must be determined by the orthonormality constraint $\left\langle \psi_{n},\phi_{m}\right\rangle =\delta_{nm}$. 

\begin{equation}
c_n= \frac{2\zeta_n}{\theta\left[\zeta_n \xi\cosh\left(\zeta\right)+(\zeta_n^{2}-\xi)\sinh\zeta_n\right]}
\end{equation}

The boundary condition which ensures flux conservation over the reflecting barrier 

\begin{equation}
\left.\frac{1}{2}g \partial_{v}\phi_{n}\right|_{v=\theta}=\left.\left(\frac{1}{2}g\partial_{v}\phi_{n}-\mu\phi_{n}\right)\right|_{v\rightarrow0^{+}}
\end{equation}

yields the characteristic equation determining the eigenvalues $\lambda_n$ of the Fokker-Planck operator 

\begin{equation}
\zeta e^{\xi}=\zeta\cosh\left(\zeta\right)+\xi\sinh\left(\zeta\right)
\end{equation}

In [ ]:
#===============================================================================
# Eigenvalues 
#===============================================================================

def CEAS(xi, n, m):
    '''Calculates analytic approximation of the characteristic equation'''
        
    i = np.complex(0, 1)
    a_n = 2*np.pi*n
   
    if xi == 0: 
        zeta = i*2*np.pi*n
        
    elif xi < 0: # noise-dominated regime: Eigenvalues are real, i.e. zeta must be purely imaginary or real                
        
        zeta = i * (a_n + 1./a_n * (1. + xi - np.exp(xi) + (-1)**m * np.sqrt((1 + xi - np.exp(xi))**2 - 2.*a_n**2*(np.exp(xi) - 1))))                 
    
    else: # drift-dominated regime: Eigenvalues are complex, i.e. zeta must have a nonzero real and complex part        
        
        zeta = xi + np.log(1 + np.sqrt(1 - np.exp(-2*xi))) + (-1)**m*i*a_n
        
    return zeta
        
def CES(xi, n, m):
    '''Finds numeric solution for characteristic equation for given n and xi'''
        
    # characteristic equation
    ce = lambda zeta: zeta*cosh(zeta) + xi*sinh(zeta) - np.exp(xi)*zeta
    # find root using analytic approximation as the start point
                
    zeta = findroot(ce, CEAS(xi, n, m), maxsteps= 100) 
        
    return zeta

def EV(xi, sig2, theta, n, m):
    '''Find numeric solution for eigenvalues determined by the characteristic equation'''
                 
    zeta = CES(xi, n, m)    
    lam_n = sig2/(2*theta**2)*(zeta**2 - xi**2)
        
    return lam_n

def EV_xi_n(theta, sig2, xi_arr, n_arr):
    '''Calculate eigenvalues for all parameters combinations in xi_arr, n_arr for given theta and sigma'''
    
    m_arr = np.array([0,1])
    
    L_xi = len(xi_arr)
    L_n  = len(n_arr)
    L_m  = len(m_arr)         
             
    lam_mat= np.zeros((L_xi, L_n, L_m), dtype = np.cfloat)

    # parameter sweep                
    for idx, t in zip(it.product(np.arange(L_xi), np.arange(L_n), np.arange(L_m)), it.product(xi_arr, n_arr, m_arr)):    
                
        xi = t[0]
        n  = t[1]
        m  = t[2]

        if xi == 0:
            zeta = - 2*np.pi**2*sig2/theta**2 * n**2
        else:                                
            lam_mat[idx[0], idx[1], idx[2]] = EV(xi, sig2, theta, n, m)
        
    return lam_mat

#===============================================================================
# Plotting 
#===============================================================================

theta = 1.
sig2  = 1.

xi_arr = np.linspace(-5, 5, 200)
n_arr = np.array([1, 2, 3])

lam_mat = EV_xi_n(theta, sig2, xi_arr, n_arr)

fig = plt.figure(figsize = (12, 8))

gs = plt.GridSpec(2,1)
ax0 = plt.subplot(gs[0]) 
ax1 = plt.subplot(gs[1]) 

for i, xi in enumerate(xi_arr):

    lam = lam_mat[i, :, :].flatten()

    xi_i = xi*np.ones(2*len(n_arr))

    ax0.plot(xi_i, np.real(lam), 'o', ms = 1.5, mfc = 'k', mec = 'k')
    ax1.plot(xi_i, np.imag(lam), 'o', ms = 1.5, mfc = 'k', mec = 'k')

# Plot horizontal lines for orientation
x = np.array([xi_arr[0], xi_arr[-1]])

for n in n_arr:

    y = -2*np.pi**2*n**2*np.array([1, 1])
    ax0.plot(x, y, c = 'k', ls = '--', lw = 0.5)

ax0.set_xlim(xi_arr[0], xi_arr[-1])    
ax0.set_ylim([0, -250])    
ax1.set_xlim(xi_arr[0], xi_arr[-1])
ax1.set_ylim([-150, 150])    

fz = 18

ax0.set_ylabel(r'RE($\lambda$)', fontsize = fz)
ax1.set_ylabel(r'IM($\lambda$)', fontsize = fz)
ax1.set_xlabel(r'$\xi$', fontsize = fz)

plt.show()

In [ ]:
#===============================================================================
# Eigenfunctions
#===============================================================================

def phi0(v, xi, eta, sig2, theta):
    '''Stationary solution of the Fokker-Planck operator for lambda = 0'''

    x = (theta - v)/theta
    c0 = (sig2/(2*eta**2)*(2*xi + np.exp(-2*xi) -1))**(-1)
    
    return c0/eta*(1-e**(-2*xi*x))
    
def phi_n(v, lam_n, eta, xi, sig2, theta):
    '''Eigenfunction corresponding to lam_n'''
    
    zeta = theta/sig2*np.sqrt(eta**2 + 2*sig2*lam_n) 
    
    cn = 2*zeta/(theta*(zeta*xi*np.cosh(zeta) + (zeta**2 - xi)*np.sinh(zeta))) 
                   
    return cn*np.exp(xi*v/theta)*np.sinh(zeta*(theta - v)/theta) 
               
#===============================================================================
# Parameter
#===============================================================================

theta = 1.
sig2 = 1.

eta_arr = np.array([-1., 1.])               
n_arr = np.array([1,2,3])

#===============================================================================
# Plotting
#===============================================================================
fz = 18 # fontsize
lfz = 14 # legend fontsize

v_arr = np.linspace(0, 1, 1000)

fig = plt.figure(figsize = (16,10))
gs = plt.GridSpec(len(eta_arr), 2)

for i, eta in enumerate(eta_arr):

    ax_i0 = plt.subplot(gs[i, 0]) 
    ax_i1 = plt.subplot(gs[i, 1]) 

    for n in n_arr:

        xi = eta*theta/sig2

        m = 0  # it is sufficient to look at m=0 because m=1 corresponds to conjugate pair
        lam_n = EV(xi, sig2, theta, n, m)
        lam_n = np.complex(lam_n.real, lam_n.imag)
        
        phi_n_i = phi_n(v_arr, lam_n, eta, xi, sig2, theta)
        ax_i0.plot(v_arr, phi_n_i.real, label = '$n=%i$' % n)
        ax_i1.plot(v_arr, phi_n_i.imag, label = '$n=%i$' % n)

        ax_i0.set_title(r'$\eta = %.2f$' % eta, fontsize = fz)

    ax_i0.plot([0,1], [0, 0], c = 'k', ls = '--', lw = 0.5)
    ax_i1.plot([0,1], [0, 0], c = 'k', ls = '--', lw = 0.5)
    ax_i0.set_xlim(0,1)
    ax_i1.set_xlim(0,1) 
    ax_i0.set_ylabel('$Re(\phi_n)$', fontsize = fz)
    ax_i1.set_ylabel('$Im(\phi_n)$', fontsize = fz)
    ax_i0.legend(fontsize = lfz)
    ax_i1.legend(fontsize = lfz)

ax_i0.set_xlabel('v', fontsize = fz)
ax_i1.set_xlabel('v', fontsize = fz)

plt.show()

We note: 

- $\phi_n(\theta) = 0$ as required by the boundary condition 
- In the drift-dominated regime ($\eta = -1.$), $\text{Im}(\phi_n(v)) \sim 0$ because $\text{Im}(\lambda_n) \sim 0$
- All $\phi_n(v)$ oscillate to ensure $\int_{v_{\text{min}}}^{\theta}dv\,\phi_{n}(v)=0$ which follows from $\psi_0(v) = 1$

In [ ]:
#===============================================================================
# Adjoint eigenfunctions
#===============================================================================

def psi_n(v, lam_n, eta, xi, sig2, theta):
               
    zeta = theta/sig2*np.sqrt(eta**2 + 2*sig2*lam_n) 
           
    return np.exp(-xi*v/theta)*(zeta*np.cosh(zeta*v/theta) + xi*np.cosh(zeta*v/theta))

fig = plt.figure(figsize = (16,10))
gs = plt.GridSpec(len(eta_arr), 2)

for i, eta in enumerate(eta_arr):

    ax_i0 = plt.subplot(gs[i, 0]) 
    ax_i1 = plt.subplot(gs[i, 1]) 

    for n in n_arr:

        xi = eta*theta/sig2

        m = 0  # it is sufficient to look at m=0 because m=1 corresponds to conjugate pair
        lam_n = EV(xi, sig2, theta, n, m)
        lam_n = np.complex(lam_n.real, lam_n.imag)

        psi_n_i = psi_n(v_arr, lam_n, eta, xi, theta, sig2)
        ax_i0.plot(v_arr, np.real(psi_n_i), label = '$n=%i$' % n)
        ax_i1.plot(v_arr, np.imag(psi_n_i), label = '$n=%i$' % n)

        ax_i0.set_title(r'$\eta = %.2f$' % eta, fontsize = fz)

    ax_i0.plot([0,1], [0, 0], c = 'k', ls = '--', lw = 0.5)
    ax_i1.plot([0,1], [0, 0], c = 'k', ls = '--', lw = 0.5)
    ax_i0.set_xlim(0,1)
    ax_i1.set_xlim(0,1) 
    ax_i0.set_ylabel('$Re(\psi_n)$', fontsize = fz)
    ax_i1.set_ylabel('$Im(\psi_n)$', fontsize = fz)
    ax_i0.legend(fontsize = lfz)
    ax_i1.legend(fontsize = lfz)

ax_i0.set_xlabel('v', fontsize = fz)
ax_i1.set_xlabel('v', fontsize = fz)

plt.show()

We note 

-  $\text{Re}(\psi_n(0,t))=\text{Re}(\psi_n(0,t))$ for all $n$

# Emission rate

## Non-interacting neurons

For non-interacting neurons and stationary input rate, the transfer function is given by 

\begin{equation}
r(t)=\Phi(\eta,\sigma^{2})+\sum_{n}f_{n}a_{n}(0)e^{\lambda_{n}t}
\end{equation}

where $\Phi(\eta,\sigma^{2})$ is stationary $r_0$ rate given by 

\begin{equation}
r_0(\eta, \sigma^2) = \frac{2\eta^{2}}{\sigma^{2}\left(2\xi-1+e^{-2\xi}\right)}.
\end{equation}

The next figure shows how $\eta$ needs to be chosen as a function of $\sigma$ to keep the $r_0$ constant.


In [ ]:
def flux_0(eta, sig2, theta):
    '''Caclulates stationary firing rate  which is equivalent to the zero mode flux over threshold'''
    xi = eta*theta/sig2
    
    return (sig2/(2*eta**2)*(2*xi + np.exp(-2*xi) -1))**(-1)

theta = 1.    

#===============================================================================
# Plot how eta needs to be adjusted for different sig2 to keep the rate constant
#===============================================================================
fz = 18
lfz = 14

plt.figure(figsize = (10, 8))
ax0 = plt.subplot(111)

r0_arr = np.array([2.5, 5., 10., 20.])    
sig2_arr = np.linspace(1.0, 9, 100)

for r0 in r0_arr:

    eta_arr = np.zeros_like(sig2_arr)

    for i, sig2 in enumerate(sig2_arr):

        eta = fsolve(lambda eta: r0 - flux_0(eta, sig2, theta), 5)[0]
        eta_arr[i] = eta

    ax0.plot(np.sqrt(sig2_arr), eta_arr, label = r'$r_0 = %i$' % int(r0))

ax0.set_xlim(1.0, 3)    
ax0.plot([0,3], [0., 0], c = 'k', ls = '--')
ax0.legend(fontsize = lfz)
ax0.set_xlabel(r'$\sigma$', fontsize = fz)
ax0.set_ylabel(r'$\eta$', fontsize = fz)

plt.show()

We note:

- If the input fluctuations $\sigma$ are larger, then a smaller average input $\eta$ is sufficient to produce the same stationary rate $r_0$
- For large enough $\sigma$, $\eta$ becomes negative and the network changes from the drift-dominated to the noise dominated regime 

We know want to investigate how the emission rate $r(t)$ relaxes to the stationary rate $r_0$ given an intital condition.  Assume all neurons are at reset $v=0$ at $t=0$ which corresponds to $\rho(v,0)=\delta(v)$. Hence,    

\begin{equation}
a_{n}(0)=\int_{v_{\text{min}}}^{\theta}dv\,\psi_{n}(v)\rho(v,0)
\end{equation}

which yields $a_{n}(0)=\psi_{n}(0)=\zeta(\lambda_n)$. The time evolution of the emission rate is given by  

\begin{equation}
r(t) = r_0 + \sum_{n}f_{n}a_{n}(0)e^{\lambda_{n}t}
\end{equation}

### Oscillatory relaxation  

The abve equation shows that oscillations can only occure if the eigenvalues $\lambda_n$ have a nonzero imaginary part, i.e. in the drift dominated regime $\eta > 0$. The characteristic decay time of the modes is given by inverse of the real part of the eigenvalues $\tau_{n}=1/\text{RE}(\lambda_{n})$. The frequency of the oscillation associated with mode $n$ is determined by the imaginary part of eigenvalue $f_{0}=\text{Im}(\lambda_{n})/2\pi\Rightarrow T_{0}=2\pi/\text{Im}(\lambda_{n})$. Hence to see prominent oscillation on the same time scale as the decay, the eigenvalues need to fulfill

\begin{equation}
T_{0}\sim\tau_{n}\Rightarrow\frac{2\pi}{\left|\text{Im}(\lambda_{n})\right|}\sim\frac{1}{\left|\text{RE}(\lambda_{n})\right|}\Leftrightarrow2\pi\left|\text{RE}(\lambda_{n})\right|\sim\left|\text{Im}(\lambda_{n})\right|
\end{equation}

The next fiugre shows the eigenvalues as a function of $\eta$ for $\sigma^2$ and $r_0 = 25$Hz.

In [ ]:
def EV_eta_sig2_n(theta, eta_arr, sig2_arr, n_arr):
    '''Calculate eigenvalues for all parameters combinations [(sig2_arr, xi_arr), n_arr] for given theta and sigma'''
        
    m_arr = np.array([0,1])
    
    L_xi = len(eta_arr)
    L_n  = len(n_arr)
    L_m  = len(m_arr)         
             
    lam_mat= np.zeros((L_xi, L_n, L_m), dtype = np.cfloat)
    
    # parameter sweep                
    for idx, t in zip(it.product(np.arange(L_xi), np.arange(L_n), np.arange(L_m)), it.product(zip(eta_arr, sig2_arr), n_arr, m_arr)):    
                
        eta  = t[0][0]
        sig2 = t[0][1]
        n    = t[1]
        m    = t[2]

        xi = eta*theta/sig2

        if xi == 0:
            zeta = - 2*np.pi**2*sig2/theta**2 * n**2
        else:                                        
            lam_mat[idx[0], idx[1], idx[2]] = EV(xi, sig2, theta, n, m)
        
    return lam_mat

r0 = 20
sig2_arr = np.linspace(1.0, 9, 100)

eta_arr = np.zeros_like(sig2_arr)

for i, sig2 in enumerate(sig2_arr):

    eta = fsolve(lambda eta: r0 - flux_0(eta, sig2, theta), 5)[0]
    eta_arr[i] = eta

xi_arr =  eta_arr*theta/sig2
n_arr = np.array([1, 2, 3])

# Get eigenvalues  
lam_mat = EV_eta_sig2_n(theta, eta_arr, sig2_arr, n_arr)

plt.figure(figsize = (10, 12))

gs =  plt.GridSpec(3,1)
ax0 = plt.subplot(gs[0]) 
ax1 = plt.subplot(gs[1]) 
ax2 = plt.subplot(gs[2]) 

for i, sig2 in enumerate(sig2_arr):

    lam = lam_mat[i, :, :].flatten()

    sig_i = np.sqrt(sig2)*np.ones(2*len(n_arr))

    ax0.plot(sig_i, np.real(lam), 'o', ms = 1.5, mfc = 'k', mec = 'k')
    ax1.plot(sig_i, np.imag(lam), 'o', ms = 1.5, mfc = 'k', mec = 'k')
    ax2.plot(sig_i, 2*np.pi*np.real(lam)/np.imag(lam), 'o', ms = 1.5, mfc = 'k', mec = 'k')
    
    
# Plot horizontal lines for orientation
x = np.array([xi_arr[0], xi_arr[-1]])

ax0.set_ylabel(r'RE$(\lambda)$', fontsize = fz)
ax1.set_ylabel(r'IM$(\lambda)$', fontsize = fz)
ax2.set_ylabel(r'$RE(\lambda)/IM(\lambda)$', fontsize = fz)
ax1.set_xlabel(r'$\sigma$', fontsize = fz)

plt.show()

The last panel in the above figure shows us that prominent ocillations occure for small $\sigma$ which requires a large $\eta$, i.e. the network is mostly driven by the drift. The time evolution of the emission rate is given by

Because eigenfunctions and eigenvalues come in conjugate pairs, we have 

\begin{equation}
r(t) = r_0 + \sum_{n>1}2\text{Re}\left(f_{n}a_{n}(0)e^{\lambda_{n}t}\right)
\end{equation}

where $a_n$, $f_n$ and $\lambda_n$ are all complex. The next figure shows the flux $f_n$ for different $n$. 

In [ ]:
def flux_n(lam_n, eta, sig2, theta):
    
    
    # sometimes the solution which comes out solver of the solve 
    # has a tiny imaginary part which causes problems    
    eps = 1.e-10
    
    lam_n_real = lam_n.real
    lam_n_imag = lam_n.imag

    if np.abs(lam_n_real) < eps:
        lam_n_real = 0.
    if np.abs(lam_n_imag) < eps:
        lam_n_imag = 0.
    
    lam_n = np.complex(lam_n_real, lam_n_imag)
    
    xi = eta*theta/sig2
    
    zeta = theta/sig2*np.sqrt(eta**2 + 2*sig2*lam_n)     
    cn = 2*zeta/(theta*(zeta*xi*np.cosh(zeta) + (zeta**2 - xi)*np.sinh(zeta))) 
    
    return 0.5*sig2*cn*zeta*np.exp(xi)

#===============================================================================
# Parameter
#===============================================================================

theta = 1.
sig2 = 1.

sig2_arr = np.array([0.5, 1.0, 1.5])**2

eta_arr = np.linspace(-4, 4, 500)

#===============================================================================
# Plotting flux 0 
#===============================================================================

fig = plt.figure(figsize = (12, 8))
ax = plt.subplot(111)

for sig2 in sig2_arr:
    
    ax.plot(eta_arr, flux_0(eta_arr, sig2, theta), label = r'$\sigma^2 = %.2f$' % sig2)

ax.legend()
ax.set_xlabel(r'$\eta$', fontsize = fz)
ax.set_ylabel(r'$f_0$', fontsize = fz)
    
plt.show()   

In [ ]:
def f_n_eta_sig2(eta_arr, sig2_arr, theta, n):
    
    f_n_mat = np.zeros((len(sig2_arr), len(eta_arr)), dtype = np.complex)
    
    for i, sig2 in enumerate(sig2_arr):        
        for j, eta in enumerate(eta_arr):
        
            xi = eta*theta/sig2
            lam_n = EV(xi, sig2, theta, n, 0)
            lam_n = np.complex(lam_n.real, lam_n.imag)        
                       
            f_n_mat[i, j] = flux_n(lam_n, eta, sig2, theta)
    
    return f_n_mat

theta = 1.
sig2 = 1.

sig2_arr = np.array([0.6, 1.0, 1.4, 1.8])**2

eta_arr_1 = np.linspace(-4., -0.1, 500)
eta_arr_2 = np.linspace(0.1, 4., 500)

f_1_mat_1 = f_n_eta_sig2(eta_arr_1, sig2_arr, theta, 1)            
f_1_mat_2 = f_n_eta_sig2(eta_arr_2, sig2_arr, theta, 1)            

fig = plt.figure(figsize = (12, 8))
gs = plt.GridSpec(2,1)

ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

for i, f_1_1 in enumerate(f_1_mat_1):           

    ax1.plot(eta_arr_1, f_1_1.real)
    ax2.plot(eta_arr_1, f_1_1.imag, label = '$\sigma^2 = %.1f$' % sig2_arr[i]) 

for i, f_1_2 in enumerate(f_1_mat_2):           

    ax1.plot(eta_arr_2, f_1_2.real, label = '$\sigma^2 = %.1f$' % sig2_arr[i])
    ax2.plot(eta_arr_2, f_1_2.imag) 
     
ax1.set_ylabel(r'RE($f_n$)', fontsize = fz)
ax1.legend()
ax2.set_xlabel('$\eta$', fontsize = fz)
ax2.set_ylabel(r'IM($f_n$)', fontsize = fz)         
ax2.legend()

plt.show()
    

In [ ]:
def emission_rate(t, n_arr, eta, sig2, theta):

    xi = eta*theta/sig2

    r = flux_0(eta, sig2, theta)
        
    for n in n_arr: 
            
        lam_n0 = EV(xi, sig2, theta, n, 0) #m=0        
        lam_n0 = np.complex(lam_n0.real, lam_n0.imag)
        
        an0 = theta/sig2*np.sqrt(eta**2 + 2*sig2*lam_n0)
        
        r_n0 = an0*flux_n(lam_n0, eta, sig2, theta)*np.exp(lam_n0*t)
        r += 2.*r_n0.real
        
    return r
        
#===============================================================================
# Parameter
#===============================================================================

theta = 1.
sig2 = 1.
r0 = 20.
eta = fsolve(lambda eta: r0 - flux_0(eta, sig2, theta), 5)[0]
xi = eta*theta/sig2

#===============================================================================
# Emission rate
#===============================================================================

# Check if eigenvalues support oscillations
lam_1 = EV(xi, sig2, theta, 1, 0)
lam_1 = np.complex(lam_1.real, lam_1.imag)

tau_1 = 1./np.abs(lam_1.real) # decay time constant
T_1 = 2*np.pi/np.abs(lam_1.imag)

print 'Ratio of the decay time and expected period of oscillation %.2f' % (tau_1/T_1) 

# Calculate emission rate
n_arr = np.array([1,2,3])
t = np.linspace(0, 5.*tau_1, 1000)

r = emission_rate(t, n_arr, eta, sig2, theta)

#===============================================================================
# Plotting
#===============================================================================
fig = plt.figure(figsize = (10, 8))
ax = plt.subplot(111)

ax.plot(t, r)
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$r$')
ax.set_xlim(left = 0.01)
ax.set_ylim([0,50])


plt.show() 
    

In [ ]:
# parameter 
h = 0.01 # efficacy
theta = 1 # threshold 
tau = 10*1e-3 
r = 1.0 # rate 
K = 10000 # number of indegrees 
r_inp = r*K
beta = 0.1# constant inhibitory current 

mu = h*tau*r_inp 
sig2 = h**2*tau*r_inp

eta = mu - beta 

print 'Sationary membrane potential: %.2f' % eta
print 'Fluctuations: %.2f' % np.sqrt(sig2)

g = sig2/tau 
h = eta/tau 

xi   = theta*h/g  
zeta = lambda x: theta/g*np.sqrt(2*x*g + h**2)

# rates need to be determined by self-consistency equation 
#sig2/2mu**2*(2*mu*theta/sig2**2 - 1 + np.exp(-2*mu*theta/sig**2))

#xi  = 0 
#eta = 0

# Leaky IF neuron
 
In this section we loosely follow [3]. We start from the Fokker-Planck planck equation 
 
\begin{equation}
\partial_t \rho(v,t)=L\rho(v,t)
\end{equation}

For a leaky intergrate-and-fire neuron, the Fokker-Planck operator is given by 

\begin{equation}
L=\partial_{v}\frac{1}{\tau}(v-\mu)+\frac{\sigma^{2}}{2\tau}\partial_{v}^{2}  
\end{equation}

where $\mu=\tau hr$ and $\sigma^{2}= \tau h^{2}r$. Hence, the Fokker-Planck planck equation can be written as  

\begin{equation}
\tau \partial_t \rho(v,t) = \left( \partial_{v}(v-\mu)+\frac{\sigma^{2}}{2\tau}\partial_{v}^{2} \right) \rho(v,t)
\end{equation}

The variable transformation 

\begin{equation}
x=\frac{\sqrt{2}}{\sigma}(v-\mu), \quad t = \frac{s}{\tau}
\end{equation}

brings the Fokker-Planck equation into a more simple form

\begin{equation}
\partial_s \rho(s,t) = \partial_x (x + \partial_x) \rho(x,s).
\end{equation}

Hence, the Fokker-Planck operator in the new variables reads 

\begin{equation}
L=\partial_x (x + \partial_x)=1+x\partial_x + \partial_x^2
\end{equation}

Our objective is to find the eigenvalues $\lambda_n$ and eigenfunctions $\phi_n(x,s)$ defined by 

\begin{equation} \label{eq: eigenfunction}
\lambda_n \phi_n(x,s)=L\phi_n(x,s)
\end{equation}

The operator $L$ can be transformed into a Hermitian operator by a similarity transformation

\begin{equation}
f(x)^{-1} L f(x) = \partial_x^2 + V(x) + c
\end{equation}

Evaluating the l.h.s. of the above equation, one finds $f(x)=exp(-x^2/4)$, $c=1/2$ and

\begin{equation}
V(x) = -\frac{1}{4}x^2 
\end{equation}

Substituting $\phi_n(x) = f(x)\varphi_n(x)$ in Eq. (\ref{eq: eigenfunction}) and multiplying from the left with $f(x)^{-1}$ yields

\begin{equation}
\lambda_{n}\varphi_{n}(x,s)=\left(\partial_{x}^{2}+V(x)+\frac{1}{2}\right)\varphi_{n}(x,s),
\end{equation}

which can be written as 

\begin{equation}
0= \varphi_{n}'' + \left(-\frac{1}{4}x^{2} + z_n \right) \varphi_{n},
\end{equation}

where we defined $z_n = 1/2 - \lambda_n$. The above differential equation is reminiscent of the Harmonic oscillator from quantum mechanism. However, in general, we expect the eigenvalues $\lambda_{n}$ to be complex, since the initial Fokker-Planck operator $L$ was not Hermitian. For real values $x$, numerically satisfactory pairs of solutions of the  above differential equation are given by the parabolic cylinder functions $U(z_n, x)$ and $V(z_n, x)$ [2]. Hence, the general solution can be written as 

\begin{equation} \label{eq: general solution}
\varphi_{n}(x,s)=c_{1}U(z_{n},x)+c_{2}V(z_{n},x)
\end{equation}

The coefficients $c_{1}$ and $c_{2}$ need to be determined by the boundary conditions similarly as we did for the PIF neuron in the previous section. In the dimension less coordinates, the boundary conditions are 

1. Absorbing boundary at the threshold: $\phi_{n}(x_{\theta}, s)=0$ with $x_{\theta}= \frac{2}{\sigma}(v_{\theta} - \mu)$
2. Flux conservation at the reset $J_n(x_\theta, s - s_0) = J_n(x^{(+)}_\text{r}, s) - J_n(x^{(-)}_\text{r}, s)$ with $x_{\text{r}}= \frac{2}{\sigma}(v_{\text{reset}} - \mu)$ 
3. Reflecting boundary at $V_\text{min}$, i.e. $J_n(x_\text{min})=0$ with $x_\text{min}= \frac{2}{\sigma}(v_\text{min} - \mu)$

Alternatively we can use a natural boundary condition for $v_\text{min} = - \infty$

3. Natural boundary condition $\phi_n(- \infty)=0$


The flux $J_n$ associated with mode $n$ is given by 

\begin{equation} 
J_n(x,s) = -x \phi_n(x,s) - \partial_x \phi_n(x,s)
\end{equation}

From the first boundary condition follows $\varphi_{n}(x_{\theta})=0$. Inserting Eq. (\ref{eq: general solution}) yields

\begin{equation} \label{eq: bc 1}
c_{2}=c_{1}\frac{U(z_{n},x_{\theta})}{V(z_{n},x_{\theta})}
\end{equation}

If we set $x_\text{min} = - \infty$, then the third boundary conditions implies that 

\begin{equation}
\phi_n(-\infty, s) = 0.
\end{equation}

From the asymptotic behavior of $U(z_n, x)$ and $V(z_n, x)$ follows 

\begin{equation}
\phi_n(-\infty, s) = c_2 \lim_{x\rightarrow-\infty} f(x)V(z_n, x) = 0
\end{equation}

which requires $c_2 = 0$ [Rethink that]. However, this would also imply $c_1 = 0$ due to Eq. (\ref{eq: bc 1}). Hence, we propose a piecewise defined solution  

\begin{equation} \label{eq: general solution 1}
\varphi_{n}(x)=\begin{cases}
a_n U(z_{n},x) & x<x_{\text{r}}\\
b_n U(z_{n},x)+c_n V(z_{n},x) & x_{\text{r}}\leq x<x_{\theta}
\end{cases}
\end{equation}

We constrain $\varphi_n(x, s)$ to be continuous at the reset $x_\text{r}$

\begin{equation}
\varphi_n(x^{(+)}_\text{r}, s) - \varphi_n(x^{(-)}_\text{r}, s) = 0.
\end{equation}

Substituting Eq. (\ref{eq: bc 1}), into the above equation yields  

\begin{equation}
b_{1} = c_{1}\left(1+\frac{U(z_{n},x_{\theta})V(z_{n},x_{\text{r}})}{V(z_{n},x_{\theta})U(z_{n},x_{\text{r}})}\right)
\end{equation}

The third boundary condition involves the flux over threshold. Substituting $\phi_n(x,s)=f(x)\varphi_n(x)$, we find 

\begin{equation}
J_{n}(x_{\theta},s)=\left.-f(x_\theta)\partial_{x}\varphi_{n}(x,s)\right|_{x=x_{\theta}} 
\end{equation}

The flux jump over reset is given by 

\begin{equation}
J_{n}(x_{\text{r}}^{(+)},s)-J_{n}(x_{\text{r}}^{(-)},s) = \left.\partial_{x}\phi_{n}(x,s)\right|_{x=x_{\text{r}}^{(-)}}-\left.\partial_{x}\phi_{n}(x,s)\right|_{x=x_{\text{r}}^{(+)}},
\end{equation}

where we used that the continuity of the eigenfunctions, i.e. $\phi_{n}(x_{\text{r}}^{(+)},s) - \phi_{n}(x_{\text{r}}^{(-)},s) = 0$. Substituting $\phi_n(x,s)=f(x)\varphi_n(x)$ and using that derivative of $f(x)$ is continuous, i.e. $\partial_x f(x_{\text{r}}^{(-)}) = \partial_x f(x_{\text{r}}^{(+)})$, yields 

\begin{equation}
J_{n}(x_{\text{r}}^{(+)},s)-J_{n}(x_{\text{r}}^{(-)},s) = f(x_{\text{r}})\left(\left.\partial_{x}\varphi_{n}(x,s)\right|_{x=x_{\text{r}}^{(-)}}-\left.\partial_{x}\varphi_{n}(x,s)\right|_{x=x_{\text{r}}^{(+)}}\right)
\end{equation}

The characteristic equation for the eigenvalues reads 

\begin{equation}
\frac{f(x_{\theta})}{f(x_{\text{r}})}\left.\partial_{x}\varphi_{n}(x)\right|_{x=x_{\theta}}=\left.\partial_{x}\varphi_{n}(x,s)\right|_{x=x_{\text{r}}^{(+)}}-\left.\partial_{x}\varphi_{n}(x,s)\right|_{x=x_{\text{r}}^{(-)}}
\end{equation}

Inserting Eq. (\ref{eq: general solution 1}) into the above equation yields    

\begin{equation}
\frac{f(x_{\theta})}{f(x_{\text{r}})}U_{x}(z_{n},x_{\theta})+\frac{U(z_{n},x_{\theta})V(z_{n},x_{\text{r}})}{V(z_{n},x_{\theta})U(z_{n},x_{\text{r}})}U_{x}(z_{n},x_{\text{r}})+\frac{U(z_{n},x_{\theta})}{V(z_{n},x_{\theta})}V_{x}(z_{n},x_{\theta})-\frac{U(z_{n},x_{\theta})}{V(z_{n},x_{\theta})}V_{x}(z_{n},x_{\text{r}}) =0
\end{equation}

TODO

The above equation can be simplified if we assume $v_\text{min} = v_\text{reset}$, because then the derivative on the r.h.s. at $x^{(-)}_\text{r}$ vanishes due to the reflecting boundary 

\begin{equation}
\frac{f(x_{\theta})}{f(x_{\text{r}})}\left.\partial_{x}\varphi_{n}(x, s)\right|_{x=x_{\theta}}=\left.\partial_{x}\varphi_{n}(x,s)\right|_{x=x_{\text{r}}^{(+)}}
\end{equation}

\begin{equation}
\left.\partial_{x}\phi_{n}(x)\right|_{x=x_{\text{min}}}=c_{1}\partial_{x}f(x)U(z_{n},x)+c_{2}\partial_{x}f(x)V(z_{n},x)
\end{equation}

The derivatives of are given by [2]

\begin{align}
\partial_{x}e^{-\frac{1}{4}x^{2}}U(z_{n},x)&=-e^{-\frac{1}{4}x^{2}}U(z_{n}-1,x) \\
\partial_{x}e^{-\frac{1}{4}x^{2}}V(z_{n},x)&=-(1/2-z_{n})V(z_{n}-1,x)
\end{align}

# Bibliography 

1.  Mattia, M., & Del Giudice, P. (2002). Population dynamics of interacting spiking neurons. Physical Review E - Statistical, Nonlinear, and Soft Matter Physics, 66(5), 1–19.

2. NIST Digital Library of Mathematical Functions. http://dlmf.nist.gov/, Release 1.0.20 of 2018-09-15. F. W. J. Olver, A. B. Olde Daalhuis, D. W. Lozier, B. I. Schneider, R. F. Boisvert, C. W. Clark, B. R. Miller, and B. V. Saunders, eds.

3. Helias, M., Tetzlaff, T., & Diesmann, M. (2018). Fluctuations and correlations in neuronal networks.
